**使用 Django 模型管理器的 annotate 方法**

从数据库取数据都是使用模型管理器 objects 的方法实现的。

获取全部分类是：**Category.objects.all()**   
假设有一个名为 test 的分类，那么获取该分类的方法是：**Category.objects.get(name='test')** 。  
objects 除了 all、get 等方法外，还有很多操作数据库的方法，而其中有一个 **annotate** 方法，该方法正可以帮我们实现本文所关注的统计分类下的文章数量的功能。

# 使用 Annotate

In [ ]:
# blog/templatetags/blog_tags.py

from django.db.models.aggregates import Count
from blog.models import Category

@register.simple_tag
def get_categories():
    # 记得在顶部引入 count 函数
    # Count 计算分类下的文章数，其接受的参数为需要计数的模型的名称
    return Category.objects.annotate(num_posts=Count('post')).filter(num_posts__gt=0)

这个 Category.objects.annotate 方法和 Category.objects.all 有点类似，它会返回数据库中全部 Category 的记录，但同时它还会做一些额外的事情，在这里我们希望它做的额外事情就是去统计返回的 Category 记录的集合中每条记录下的文章数。代码中的 Count 方法为我们做了这个事，它接收一个和 Categoty 相关联的模型参数名（这里是 Post，通过 ForeignKey 关联的），然后它便会统计 Category 记录的集合中每条记录下的与之关联的 Post 记录的行数，也就是文章数，最后把这个值保存到 num_posts 属性中。

# 在模板中引用新增的属性

```HTML
templates/base.html

<ul>
  {% for category in category_list %}
  <li>
    <a href="{% url 'blog:category' category.pk %}">{{ category.name }}
      <span class="post-count">({{ category.num_posts }})</span>
    </a>
  </li>
  {% empty %}
  暂无分类！
  {% endfor %}
</ul>
```